# 1.Sequence-to-Sequence
<br>

## 1.1 encoder(부호기)-decoder(복호기) 구조

<img src="./image/부호기복호기.jpg" width="500" height="200">
seq2seq 신경망은 두 개의 모듈식 순환 신경망으로 구성되는데 두 순환 신경망 사이에는 생각 벡터가 끼어 있다. **부호기는 주어진 입력 순차열을 하나의 생각 벡터로 변환한다. 복호기는 그 생각 벡터에 기초해서 일련의 토큰들을 생성한다.** 

<부호기>

부호기-복호기 모형의 처음 절반은 순차열 부호기이다. 이 신경망은 자연어 텍스트 같은 순차열을 그보다 낮은 차원의 표현, 이를테면 생각벡터 같은 표현으로 부호화한다. 사실 LSTM은 가변 길이 텍스트의 부호기로 사용하기에 아주 적합하다. 장단기 기억망은 텍스트의 의미를 생각 벡터라고 부르는 내부 표현으로 부호화한다. 케라스 LSTM 모형을 추가할 때, return_state=True를 지정하면 은닉층 상태가 출력에 포함된다. 그 상태 벡터가 바로 부호기의 출력이다. 그것을 복호기에 입력하면 된다. 부호기가 산출하는 생각 벡터는 두 부분으로 구성된다. **하나는 부호기 은닉층이 출력한 활성값들의 벡터이고, 다른 하나는 입력 순차열에 대한 LSTM 세포의 기억 상태이다.** 

<복호기>

나머지 절반은 순차열 복호기이다. 복호기는 주어진 벡터를 사람이 읽을 수 있는 텍스트로 복원하는 역할을 한다. 부호기에서 생성된 생각 벡터는 복호기 망의 초기 상태로 쓰인다. 복호기는 그 초기 상태와 특별한 종류의 입력인 시작 토큰을 출발점으로 삼아서 목표 순차열의 첫 요소(문자 또는 단어)를 생성하고, 그 요소에 기초해서 둘째 요소를 생성하는 과정을 반복해서 하나의 목표 순차열을 산출한다. 

<부호기-복호기 훈련 과정>

**훈련 시에는 부호기에 입력한 순차열(번역의 경우 원문)에 대한 기대 순차열(번역의 경우 번역문)도 복호기의 입력에 포함된다.** 생각 벡터와 시작 요소에 기초해서 **복호기가 예측한 토큰과 기대 순차열에 있는 해당 위치의 토큰 차이에 기초한 오차를 역전파**함으로써 복호기는 생각 벡터를 적절한 순차열로 복원하는 방법을 배우게 된다. 
<img src="./image/부호기복호기2.jpg" width="500" height="200">

## 1.2 대화 생성을 위한 seq2seq
<br>
기계번역이 챗봇 문제와 무슨 관련이 있냐고 생각할 수도 있지만, 둘은 상당히 비슷하다. 번역과 대화 모두, 모형을 한 순차열을 다른 순차열로 사상해야 한다.  **기계번역 시스템이 영어 토큰들로 이루어진 순차열을 독일어 토큰들로 이루어진 순차열로 사상하는 것은 챗봇이 대화 과정에서 사용자의 자연어 문장을 적절한 기대 응답으로 사상하는 것과 대단히 비슷하다** 물론 챗봇은 남의 말을 그대로 따라하는 앵무새가 아니라 사용자의 말에 좀 더 적절히 반응하는 지능적인 존재이어야 한다. 그러려면 **번역의 경우보다 더 많은 훈련 자료와 더 높은 차원의 생각 벡터가 필요하다.** 훈련 집합은 예상 가능한 사용자 질문과 그에 대한 적절한 응답문으로 이루어진다. 이때 모형의 구조 자체는 기계 번역을 위한 것과 다르지 않다. 원문과 번역문 대신 질문과 응답으로 훈련하는 것일 뿐이다. 케라스는 seq2seq 신경망 구축을 위한 부호기-복호기 모듈을 제공한다. 또한 번역이나 대화, 심지어는 유전형-표현형 변환 문제를 푸는데 필요한 LSTM 망의 내부 정보에 접근하는 API도 제공한다. 

## 1.3 정리
<br>
seq2seq모형은 두개의 신경망으로 구성된다. 하나는 생각 벡터를 생성하는 부호기이고 다른 하나는 그 생각 벡터를 초기 상태로 삼아서 순차열을 생성하는 복호기이다. 복호기는 초기 상태와 시작 토큰으로부터 출력 순차열의 첫 요소를 생성한다. 그런 다음에는 그 요소를 씨앗삼아서 그 다음 요소를 생성한다. 이런 과정을 중지 토큰이 나오거나 최대 순차열 길이에 도달할 때까지 반복한다. 그때까지의 요소들로 이루어진 순차열이 모형 전체의 기대 출력이다. 훈련 시에는 이 순차열을 목표 순차열과 비교해서 오차를 역전파하고, 추론(신경망 실행)시에는 이 순차열을 사용자에게 제시한다. 
<br><br>

# 2. Seq2Seq NLP 파이프라인 구축

이제부터는 이전 장들에서 이미 배운 부품들을 이용해서 seq2seq 기계 학습 파이프라인을 실제로 구성해본다. 

## 2.1 자료 집합 준비

이전 예제들 처럼 이번에도 입력 자료의 길이를 일정하게 맞춘다. 입력 견본들의 정답에 해당하는 목표 견본들 역시 마찬가지로 특정 길이로 정리한다. 이때 입력 순차열과 목표 순차열의 길이가 같아야 할 필요는 없다. **목표 순차열들에는 특별한 시작 토큰과 중지 토큰도 배정해야 한다.** 복호기는 이 토큰들을 이용해서 문장 생성의 시작과 끝을 인식한다. 이전 장들에서 훈련 집합은 입력과 기대 출력의 쌍들로 구성되었다. **seq2seq 모형에서 훈련 집합은 세 개의 요소로 이루어진다. 하나는 입력 순차열이고 다른 하나는 시작 토큰으로 시작하는 기대 출력, 나머지 하나는 시작 토큰 없는 기대 출력이다.** 

## 2.2 케라스의 seq2seq 모형

훈련과정에서는 부호기와 복호기를 함께 훈련한다. 각 훈련 견본은 부호기에 입력되는 입력 순차열, 복호기에 입력되는 입력 순차열, 그리고 복호기의 바람직한 출력 문자열로 구성된다. 복호기에 두개의 순차열이 필요한 이유는, 복호기가 제공한 초기 상태(생각 벡터)와 입력 순차열의 토큰들을 이용해서 앞에서 본 토큰 대 토큰 예측 방식으로 출력 순차열을 생성하기 때문이다. **따라서 복호기의 입력 순차열과 출력 순차열은 사실 같은 것이다. 단지 토큰들이 시간 단계 하나 만큼이나 어긋나 있을 뿐이다.** 

## 2.3 순차열 부호기

부호기의 유일한 목적은 생각 벡터를 생성하는 것이다. 이 부호기만 따로 훈련할 수는 없다. **부호기가 적절한 생각 벡터를 산출하도록 훈련하는 데 필요한 역전파는 모형의 나머지 절반에서 복호기가 생성한 순차열에 대한 오차에서부터 출발한다.** 하지만 부호기와 복호기는 개별적인 모듈이고, 다른 부호기나 복호기로 바꾸어 끼는 것도 가능하다. 예를 들어 영어를 독일어로 번역하기 위해 훈련한 부호기를 영어-스페인어 번역을 위한 부호기로 사용하는 것도 가능하다. 
<img src="./image/부호기.jpg" width="500" height="200">
위의 그림은 두 내부 상태가 생성되는 방식을 보여준다. 부호기는 각 시간 단계에서 은닉 상태와 기억 상태를 갱신한다. 마지막 시간 단계의 두 상태는 복호기의 초기 상태로 쓰인다. 

In [ ]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(num_neurons, return_state=True)
# LSTM의 내부 상태를 얻으려면 이 return_state를 True로 설정해야 한다. 
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# lstm층의 세 반환값 중 첫번재 것은 그 층의 출력이다. 
# return_state를 true로 설정해서 부호기는 이 층의 출력과 함게 
#마지막 시간 단계의 은닉 상태들(state_h)과 기억 상태(state_c)도 돌려준다. 
# 복호기를 위한 생각 벡터는 층의 출력을 제외한 state_h와 state_c로 구성된다. 
encoder_states = [state_h, state_c]

## 2.4 생각 벡터 복호기

복호기도 부호기 만큼 구성이 간단하다. 주된 차이는, 각 시간 단계에서 신경망의 예측 결과를 출력한다는 것이다. 그 출력(다음 토큰)을 콕표 순차열의 다음 토큰과 비교해서 예측의 정확성을 판정한다. **복호기는 부호기가 산출한 생각 벡터를 초기 상태로 삼아서 토큰 대 토큰 방식으로 다음 토큰을 예측한다** **복호기는 주어진 생각 벡터와 입력 순차열의 토큰들로부터 입력 순차열을 정확히 복원하는 방법을 배워야 한다** 

훈련과정에서는 부호기가 출력한 순차열을 하나의 밀집층dense layer에 입력한다. 그 밀집층은 모든 가능한 출력 토큰의 개수와 동일한 수의 뉴런으로 이루어지며, 활성화 함수로는 그 모든 토큰에 대한 소프트맥스 함수를 사용한다. **따라서 각 시간 단계에서 밀집층은 하나의 확률분포를 출력하는데, 그 확률분포는 모든 가능한 토큰 각각에 대해 그것이 다음 토큰이 될 확률들로 이루어져 있다.** seq2seq모형에서는 부호기에 입력될 수 있는 서로 다른 토큰들의 수(input_vocab_size)와 출력 문장에 사용할 수 있는 서로 다른 토큰들의 수(output_vocab_size)가 달라도 된다는 점을, **즉 부호기의 어휘 크기와 복호기의 어휘 크기가 같을 필요가 없다는 점을 주목하자.** 

In [ ]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(num_neurons, return_sequences=True, return_state=True)
# 부호기를 만들때처럼 LSTM층을 정의한다. 
# return_sequences도 True로 설정한다.
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
# 케라스의 함수형 API를 이용해서 부호기의 내부 상태들(생각벡터)이 복호기의 초기상태가 되도록 정의한다.
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
# 소프트맥스 층은 모든 가능한 토큰의 다음 토큰 확률들로 이루어진 확률분포를 산출한다.
decoder_outputs = decoder_dense(decoder_outputs)

## 2.5 seq2seq 신경망 조립

케라스의 함수형 API를 이용하면 객체를 함수처럼 호출해서 모형을 조립할 수 있다. Model객체는 신경망의 입력 부분과 출력 부분을 지정하는 수단을 제공한다. 밑의 모델을 보면 입력이 두가지이다. 위에서 부호기를 위한 입력 하나와 복호기를 위한 입력 하나를 정의했다. 이 두 입력은 세 가지 요소로 이루어진 훈련 견본의 처음 두 요소에 대응된다. 출력층으로는 위에서 정의한 decoder_outputs를 지정한다. 여기에는 앞에서 정의한 모형 설정 전체가 포함되어 있다. 

In [ ]:
model = Model(inputs=[encoder_inputs, decoder_inputs], 
              outputs = decoder_outputs)
# 입력이 여러 개인 경우에는 이처럼 inputs인수에 입력들의 목록을 저장하면 된다. 
#ouput인수도 출력들의 목록을 지원한다. 

## 2.6 seq2seq2 신경망 훈련

이번 순차열 생성은 범주형 분류 문제에 해당한다. 즉, 여기서 범주들은 사용 가능한 모든 토큰에 해당하며, 따라서 범주가 상당히 많다. 이진 상태가 아니라 다수의 문자 도는 단어를 예측하므로, 신경망이 최적화할 손실함수로는 binary_crossentropy대신 범주형 교차 엔트로피 categorical_crossentropy를 지정해야 한다. 

seq2seq 모형의 훈련은 상당히 많은 양의 계산이 필요하다. 그렇기 때문에 GPU 활용을 강력히 추천한다. GPU를 이용하면 훈련 속도가 30배 정도 빨라진다. 합성곱 신경망과는 달리 LSTM은 병렬성이 그리 좋지 않으므로, GPU를 최대한 활용하려면 LSTM층을 GPU에 특화된 CUDA기반의 CuDNNLSTM층으로 대체해야 한다. 

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs)
# 위의 모형 정의에 따라 두 개의 입력 자료를 지정한다. 
# 훈련과정에서 첫 입력 자료는 부호기에, 둘째 입력 자료는 복호기에 전달된다. 


<br><br>

# 3. Seq2Seq 신경망을 이용한 챗봇 구축

이번 예제의 챗봇은 문자 수준 seq2seq 신경망으로 구현된다. 이 챗봇의 훈련에는 코넬 대학교의 영화 대화 말뭉치를 사용한다. 이 챗봇 예제는 케라스 블로그에 나온 seq2seq 예제에 기초한 것이다. 

## 3.1 훈련 자료 준비

In [13]:
import os
import warnings
warnings.filterwarnings('ignore')
from nlpia.loaders import get_data, DATA_PATH

In [14]:
df = get_data('moviedialog')
input_texts, target_texts = [], []  # 입력 문장과 목표 문장들을 담는다.
input_vocabulary = set()  # 이 두 집합은 입력 문장들과 목표 문장들에 등장한 문자들을 담는다.
output_vocabulary = set()
start_token = '\t'
stop_token = '\n'
# 목표 순차열에는 시작 토큰과 중지 토큰이 붙는다. 
# 이 문자들은 보통의 문장에는 쓰이지 않는, 오직 시작 토큰과 중지 토큰으로만 쓰이는 문자들이어야 한다.
max_training_samples = min(25000, len(df) - 1)  
# 훈련에 사용할 견본의 수이다. 
# 미리 정의한 최댓값과 말뭉치 파일에서 불러온 행 수 중 더 작은 것을 사용한다. 

for input_text, target_text in zip(df.statement, df.reply):
    target_text = start_token + target_text \
        + stop_token  
    # target_token을 시작토큰과 중지 토큰으로 감싸야 한다. 
    input_texts.append(input_text)
    target_texts.append(target_text)
    
    for char in input_text: 
        # input_texts에 등장한 중복 없는 문자들의 집합을 만든다. 
        if char not in input_vocabulary:
            input_vocabulary.add(char)
    for char in target_text:
        if char not in output_vocabulary:
            output_vocabulary.add(char)

INFO:nlpia.futil:Reading CSV with `read_csv(*('C:\\Users\\today\\anaconda3\\lib\\site-packages\\nlpia\\data\\moviedialog.csv',), **{'nrows': None, 'low_memory': False})`...


In [15]:
df

,statement,reply
0,you're asking me out. that's so cute. what's y...,forget it.
1,"no, no, it's my fault we didn't have a proper ...",cameron.
2,"gosh, if only we could find kat a boyfriend...",let me see what i can do.
3,c'esc ma tete. this is my head,right. see? you're ready for the quiz.
4,how is our little find the wench a date plan p...,"well, there's someone i think might be"
...,...,...
64345,lighting coghill' 5 cigar: our good colonel du...,um. there are rumours that my lord chelmsford ...
64346,do you think she might be interested in someone?,which one?
64347,choose your targets men. that's right watch th...,keep steady. you're the best shots of the twen...
64348,colonel durnford... william vereker. i hear yo...,"good ones, yes, mr vereker. gentlemen who can ..."


## 3.2 문자 사전 구축

이전 장들의 예제와 비슷하게, 신경망에는 입력 텍스트와 목표 텍스트의 실제 문자들이 아니라 그 문자들을 부호화한 원핫 벡터들을 입력해야 한다. **원핫 벡터를 생성하려면 각 문자를 적절한 벡터 성분 색인으로 사상하는 토큰 사전이 필요하다.** 그러한 토큰 사전을 입력 텍스트와 목표 텍스트에 대해 따로 만들어야 한다. 또한 그 반대 방향으로의 사상(색인을 문자에 대응시키는)을 위한 역사전도 만들어야 한다. 이 역사전을 이용해서 복호기가 산출한 색인을 문자로 변환한다. 

In [16]:
# 사전 생성을 위해 앞에서 구한 문자 집합의 문자들을 알파벳순으로 정렬한다. 
input_vocabulary = sorted(input_vocabulary)  
output_vocabulary = sorted(output_vocabulary)

# 입력 텍스트와 목표 텍스트의 고유한 문자 개수를 얻는다. 
# 이들은 이후 원핫 부호화를 위한 행렬에 쓰인다. 
input_vocab_size = len(input_vocabulary)  
output_vocab_size = len(output_vocabulary)

# 입력 자료와 목표 자료의 최대 문장 길이(최대 순차열 토큰 수)도 구한다. 
max_encoder_seq_length = max(
    [len(txt) for txt in input_texts]) 
max_decoder_seq_length = max(
    [len(txt) for txt in target_texts])

# input_vocabulary의 문자를 색인으로 사상하는 사전들을 생성한다. 
# 이들은 문자를 원핫 벡터로 변환하는데 쓰인다. 
input_token_index = dict([(char, i) for i, char in enumerate(input_vocabulary)])
target_token_index = dict([(char, i) for i, char in enumerate(output_vocabulary)])

# 사전들을 거꾸로 훑어서 역사전들을 만든다. 
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())  # <5>
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [18]:
print(input_token_index)
print(reverse_input_char_index)

{' ': 0, '!': 1, "'": 2, ',': 3, '.': 4, '0': 5, '1': 6, '2': 7, '3': 8, '4': 9, '5': 10, '6': 11, '7': 12, '8': 13, '9': 14, ':': 15, ';': 16, '?': 17, 'a': 18, 'b': 19, 'c': 20, 'd': 21, 'e': 22, 'f': 23, 'g': 24, 'h': 25, 'i': 26, 'j': 27, 'k': 28, 'l': 29, 'm': 30, 'n': 31, 'o': 32, 'p': 33, 'q': 34, 'r': 35, 's': 36, 't': 37, 'u': 38, 'v': 39, 'w': 40, 'x': 41, 'y': 42, 'z': 43}
{0: ' ', 1: '!', 2: "'", 3: ',', 4: '.', 5: '0', 6: '1', 7: '2', 8: '3', 9: '4', 10: '5', 11: '6', 12: '7', 13: '8', 14: '9', 15: ':', 16: ';', 17: '?', 18: 'a', 19: 'b', 20: 'c', 21: 'd', 22: 'e', 23: 'f', 24: 'g', 25: 'h', 26: 'i', 27: 'j', 28: 'k', 29: 'l', 30: 'm', 31: 'n', 32: 'o', 33: 'p', 34: 'q', 35: 'r', 36: 's', 37: 't', 38: 'u', 39: 'v', 40: 'w', 41: 'x', 42: 'y', 43: 'z'}


## 3.3 원핫 부호화 훈련 집합 생성

고유한 입력 문자 개수가 n이라 할때, 각 문자는 하나의 n차원 행벡터가 된다. 각 견본의 모든 문자 원핫 벡터를 층층이 샇아서 행렬을 만들고, 그런 행렬들을 모아서 하나의 훈련 텐서를 만든다. 목표 문장들에도 마찬가지로 처리한다. 

In [19]:
import numpy as np  

# (견본 개수, 최대 순차열 길이, 고유한 토큰 개수)형태로 훈련 텐서들을 0으로 초기화
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, input_vocab_size),dtype='float32') 
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, output_vocab_size),dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, output_vocab_size),dtype='float32')

# 훈련 견본들을 훑는다.입력 텍스트와 목표 텍스트가 짝을 이루어야 하니까 하나의 루프로 처리
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    # 각 견본의 각 문서를 원핫 벡터로 변환한다. 
    for t, char in enumerate(input_text):  
        # 현재 문자에 해당하는 성분만 1로 설정. 나머지는 모두 0
        # encoder_input_data는 입력견본들의 원핫 표현들이 담게 된다.
        encoder_input_data[i, t, input_token_index[char]] = 1. 
    
    # 복호기의 입력 자료인 decoder_input_data도 같은 방식으로 부호화
    # decoder_input_data보다 시간 단계 하나만큼 뒤처진 목표 자료 집합decoder_target_data도 만든다.
    for t, char in enumerate(target_text):  
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1

In [23]:
print(len(encoder_input_data[0]))
print(len(encoder_input_data[0][0]))

# 입력 견본은 한 견본당 100개의 토큰으로 이루어져 있고
# 각 토큰은 44개 백터값으로 구성되어진다. (총 char의 개수가 44개)

100
44


## 3.4 Seq2Seq 챗봇의 훈련

In [29]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense 

batch_size = 64    # 견본 64개를 하나의 배치로 사용해서 훈련. 배치크기를 늘리면 훈련이 빨라지지만. 메모리 요구량이 높아질 수 있다. 
epochs = 1      # seq2seq를 훈련하려면 꽤 많은 반복이 필요하다. 100으로 하면 더 좋지만 지금은 시간이 없어서..
num_neurons = 256  # LSTM층의 뉴런 수를 256으로 설정한다.

encoder_inputs = Input(shape=(None, input_vocab_size))
encoder = LSTM(num_neurons, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]


decoder_inputs = Input(shape=(None, output_vocab_size))
decoder_lstm = LSTM(num_neurons, return_sequences=True,return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,initial_state=encoder_states)
decoder_dense = Dense(output_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['acc'])
model.fit([encoder_input_data, decoder_input_data],
          decoder_target_data, batch_size=batch_size, epochs=epochs, validation_split=0.1) 

Train on 57915 samples, validate on 6435 samples
57915/57915 [==============================] - 2020s 35ms/sample - loss: 0.7536 - acc: 0.1219 - val_loss: 0.6449 - val_acc: 0.1555


3시 24분 훈련 시작 - 3시57분 끝

## 3.5 순차열 생성을 위한 모형 설정

In [34]:
## 출력 순차열 생성을 위한 부호기 모형
# 이전에 정의한 encoder_input와 encoder_states를 사용한다. 
# 이 encoder_model 모형에 대해 predict 메서드를 호출하면 생각 벡터가 반환된다. 
encoder_model = Model(encoder_inputs, encoder_states)


## 생각 벡터로부터 순차열을 생성하는 모형
# 부호기가 생성한 생각 벡터를 입력받기 위한 정의이다. 
thought_input = [Input(shape=(num_neurons,)), Input(shape=(num_neurons,))]


# 다음 코드에 의해 복호기의 LSTM 층은 생각벡터를 초기 상태로 사용하게 된다. 
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=thought_input)
# LSTM 층의 세 출력 중 마지막 두개(은닉상태와 기억 상태)로 LSTM 층의 출력을 산출하는 밀집층을 정의한다.
decoder_states = [state_h, state_c]
# LSTM층의 출력을 밀집층으로 전달해서 다음 토큰을 예측한다. 
# 모든 가능한 토큰의 다음 토큰 확률들로 이루어진 확률분포를 출력한다. 
# 이 확률분포는 훈련 과정에서 본 모든 토큰의 공동 출현 빈도에 기초한 것이다. 
decoder_outputs = decoder_dense(decoder_outputs)


# 이제 앞의 정의들로 하나의 복호기 모형을 정의한다. 
decoder_model = Model(
    # decoder_inputs와 생각벡터의 조합이 이 복호기 모형의 입력이다. 
    [decoder_inputs] + thought_input,
    [decoder_outputs] + decoder_states)
    # 모형의 출력은 밀집층의 출력과 갱신된 상태의 조합이다. 

## 3.6 순차열 생성(예측)및 출력

다음 함수는 주어진 입력 순차열(입력 문장의 원핫 벡터들로 이루어진)을 앞에서 훈련한 seq2seq 신경망 부호기에 넣어서 생각 벡터를 얻고, 그 생각 벡터를 복호기에 넣어서 적절한 순차열을 생성한다. 

In [44]:
def decode_sequence(input_seq):
    thought = encoder_model.predict(input_seq)  # 복호기에 입력할 생각 벡터를 얻는다.

    target_seq = np.zeros((1, 1, len(output_vocabulary)))       # 훈련과 달리 target_seq는 시작 토큰이 없는 0텐서로 시작한다.
    target_seq[0, 0, output_vocabulary.index(stop_token)] = 1.  # 복호기의 첫 입력 토큰은 시작 토큰이다. 
    stop_condition = False
    generated_sequence = ''

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + thought) 
        # 생성된 토큰과 갱신된 생각 벡터를 복호기에 넣어서 다음 토큰을 생성(예측)한다. 

        generated_token_idx = np.argmax(output_tokens[0, -1, :])
        generated_char = output_vocabulary[generated_token_idx]
        generated_sequence += generated_char
        if (generated_char == stop_token or
                len(generated_sequence) > max_decoder_seq_length):  # 생성 과정을 끝낼 조건이 충족되면 stop_condition을 true로 설정한다
            stop_condition = True

        target_seq = np.zeros((1, 1, len(output_vocabulary)))  # 목표 순차열을 갱신하고 이번 시간 단계에서 생성한 토큰을 다음 시간 단계의 입력으로 설정한다. 
        target_seq[0, 0, generated_token_idx] = 1.
        thought = [h, c]  # 생각 벡터 갱신

    return generated_sequence

아래 responce() 함수는 사용자의 입력 문장에 대한 응답문을 출력하는 함수이다. 
- 이 함수는 먼저 주어진 입력 문장을 적절한 원핫 벡터들의 순차열로 변환해서 앞의 decoder_sequence()함수를 호출한다.
- deocder_sequence()는 그 입력 순차열을 생각 벡터로 변환
- 그 생각 벡터에 기초해서 생성한 응답문(문자들의 순차열)을 돌려준다. 
- respond는 그 응답문을 적절한 형식으로 출력한다. 

In [42]:
def respond(input_text):
    input_text = input_text.lower()
    input_text = ''.join(c if c in input_vocabulary else ' ' for c in input_text)
    input_seq = np.zeros((1, max_encoder_seq_length, len(input_vocabulary)), dtype='float32')
    
    
    # 입력 텍스트의 각 문자를 원핫 벡터로 변환해서 하나의 원핫 벡터를 생성한다.
    # 이후 부호기는 이 입력 텐서로부터 생각 벡터를 생성한다. 
    for t, c in enumerate(input_text):
        input_seq[0, t, input_vocabulary.index(c)] = 1.
    
    # decoder_sequence 함수는 훈련된 모형을 이용해서 생성한 응답 순차열을 돌려준다. 
    decoded_sentence = decode_sequence(input_seq)
    print('Human: {}'.format(input_text))
    print('Bot:', decoded_sentence)
    return decoded_sentence

In [46]:
respond('Hi Rosa, how are you?')
respond('Hi Jim, how are you?')
respond('Hi Barak, how are you?')

Human: hi rosa, how are you?
Bot: i don't know what i was the pack on the mane and and the sane the pack on the pack on the sare the pack
Human: hi jim, how are you?
Bot: i don't know what i was the pack on the mane and and the sane the pack on the pack on the sare the pack
Human: hi barak, how are you?
Bot: i don't know what i was the pack on the mane and and the sane the pack on the pack on the sare the pack


"i don't know what i was the pack on the mane and and the sane the pack on the pack on the sare the pack"

# 4. 개선안

잘 설계된 교과 과정이 학습에 도움이 된다. 훈련 자료를 여러 부분으로 나누고 적절한 순서로 투입하면, 학습에 중요한 부분에 신경망이 주의를 집중하게 하면, 신경망이 학습 내용을 좀 더 빠르게 소화할 수 있다. 


## 4.1 버키팅을 이용한 학습 복잡도 감소
<br>
서로 다른 길이의 입력 순차열들을 일정한 길이로 정리할 때는 짧은 순차열에 다수의 채움 토큰을 추가하는 방법이 흔히 쓰인다. 그런데 이런 토큰 채우기 방법을 사용하면 훈련을 위한 계산 비용이 높아질 수 있다. 이에 대한 해결책 하나가 버키팅 bucketing 통 분류이다. 버키팅에서는 순차열들을 길이에 따라 적절한 그룹(버킷)들로 분류한다. 예를 들어 첫 버킷에는 길이가 5에서 10 토큰인 순차열들을 담고, 둘째 버킷에는 길이가 11에서 15토큰인 순차열들을 담는 식이다. 각 버킷을 하나의 배치로 삼고 순차열들에 버킷 최대 길이가지만 채움 토큰들을 추가해서 신경망을 훈련하면 너무 많은 채움 토큰들 때문에 생기는 비효율성을 피할 수 있다. 몇몇 심층 학습 프레임워크들은 주어진 입력 자료를 분석해서 최적의 버킷 크기들을 제시하는 버키팅 도구를 제공한다. 
<img src="./image/버키팅.jpg" width="500" height="200">

## 4.2 주의 메커니즘
<br>

LSA와 마찬가지로 입력 문서가 길수록 생각 벡터의 품질이 나빠진다. 즉, 긴 문서에 대한 생각 벡터는 그 문서를 그리 정확하게 대표하지 못한다. 생각 벡터의 차원수는 LSTM 층의 차원수(뉴런수)로 한정된다. 블로그 글이나 뉴스 기사 같은 긴 문서를 요약하도록 seq2seq 모형을 훈련해야 한다면 문제가 생길 수 있다. 이처럼 긴 문서를 읽고 그 문서에서 가장 중요한 정보를 파악하도록 신경망을 훈련하기는 쉽지 않은 일이다. **어떤 문서의 표제나 요약문(해당 생각 벡터)을 산출하려면 문서의 모든 부분을 동일하게 취급해서 그 모든 의미를 대표하려 들기보다는 문서의 특정 측면 또는 특정 부분에 초점을 두는 것이 바람직하다** 바다나우 조경현, 벤지오는 2015년 ICLR에서 이 문제에 대한 하나의 해법을 제시했다. 그들이 제시한 것은 주의 메커니즘이다. **이 해법은 입력 순차열 중 어디에 주의를 두어야 할 것인지를 복호기에 알려준다. 더 구체적으로 말하면, 부호기는 전체 입력 순차열의 열지도(heatmap)에 비유할 수 있는 정보를 학습한다. 부호기가 각 시간 단계마다 생성한 열지도는 복호기에도 전달된다. 복호기는 순차열의 특정 부분을 복원할 때 생각 벡터뿐만 아니라 그 부분이 얼마나 뜨거운지에 관한 정보도 참고한다** 주요 개념이 순차열의 어디에 있든, 주의 메커니즘은 그 부분을 명시적으로 강조함으로써 신경망이 해당 부분을 좀 더 잘 반영하는 생각 벡터를 만들어내게 한다. 

주의 메커니즘을 구성하고 조율하기는 쉽지 않지만, 여러 심층 학습 프레임워크는 주의 메커니즘을 도입하는데 도움이 되는 구현들을 제공한다. 

# 요약

- 재사용 가능한 모듈식 encoder-decoder 구조로 seq2seq 신경망을 구축할 수 있다. 
- 부호기는 생각 벡터를 산출한다. 생각 벡터는 가변길이 입력 순차열의 정보를 표현하는 고정 차원 밀집 벡터이다. 
- 복호기는 생각 벡터를 이용해서 출력 순차열을 예측한다. 챗봇의 응답문이 그러한 출력 순차열에 해당한다. 
- 부호기와 복호기 사이의 생각 벡터 덕분에, 입력 순차열과 출력 순차열 길이가 같을 필요가 없다. 
- 생각 벡터에 담는 정보의 양에는 한계가 있다. 좀 더 복잡한 개념들을 생각 벡터로 부호화해야 한다면, 주의 메커니즘을 이용해서 중요한 정보를 선택적으로 생각 벡터에 담는 것이 하나의 해결책이다. 